<a href="https://colab.research.google.com/github/yeswanthraj007/Recommendation-system-collaborative-filtering-/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this to upload files from the desktop "Remove the three apostrophes before running the code."
'''from google.colab import files
uploaded = files.upload()'''
# Run this for installing turicate library since its not been installed in the colab initially.
!apt install libnvrtc8.0
!pip uninstall -y mxnet-cu80 && pip install mxnet-cu80==1.1.0
!pip install turicreate


In [15]:

# Importing necessary libraries
import pandas as pd
import numpy as np
import turicreate
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances 
# Reading the file
i_cols = ['user_id', 'post_id', 'timestamp']
views = pd.read_csv('/content/views.csv')

u_cols = ['_id', 'name', 'gender', 'academics']
users = pd.read_csv("/content/users.csv",)


r_cols = ['_id', 'title', 'category', 'post_type']
posts = pd.read_csv('/content/posts.csv')
# After loading the dataset, we should look at the content of each file (users, ratings, items).

# Looking at the user file
print("\nUsers Data :")
print("shape : ", users.shape)
print(users.head())

# We have 943 users in the dataset and each user has 5 features, i.e. user_ID, age, sex, occupation and zip_code. Now let’s look at the ratings file.

# Posts Data
print("\nPosts Data :")
print("shape : ", posts.shape)
print(posts.head())

# We have 100k ratings for different user and movie combinations. Now finally examine the items file.

# Views Data
print("\nViews Data :")
print("shape : ", views.shape)
print(views.head())



Users Data :
shape :  (118, 4)
                        _id                  name gender      academics
0  5d60098a653a331687083238  Nivesh Singh Chauhan   male  undergraduate
1  5d610ae1653a331687083239         Gaurav Sharma   male       graduate
2  5d618359fc5fcf3bdd9a0910         Akshay Mishra   male  undergraduate
3  5d6d2bb87fa40e1417a49315        Saksham Mathur   male  undergraduate
4  5d7c994d5720533e15c3b1e9         Varun Chowhan   male  undergraduate

Posts Data :
shape :  (493, 4)
                        _id  ...  post_type
0  5d62abaa65218653a132c956  ...       blog
1  5d6d39567fa40e1417a4931c  ...       blog
2  5d7d23315720533e15c3b1ee  ...       blog
3  5d7d405e5720533e15c3b1f3  ...    artwork
4  5d80dfbc6c53455f896e600e  ...       blog

[5 rows x 4 columns]

Views Data :
shape :  (1449, 3)
                    user_id                   post_id                 timestamp
0  5df49b32cc709107827fb3c7  5ec821ddec493f4a2655889e  2020-06-01T10:46:45.131Z
1  5ed3748576027d35905cca

In [ ]:
# We can try splitting data in this way alternatively.
'''#split into inputs and outputs
data = views.values
X = data[:, :-1]
y = data[-1:, :]
print("\n The split out input and output elements are: ", X.shape, y.shape)
# split into train test sets

X_train, X_test = train_test_split(X, test_size=0.33, random_state=1)
print("\n The elements sorted out into train and test sets are: ", X_train.shape, X_test.shape)'''

'#split into inputs and outputs\ndata = views.values\nX = data[:, :-1]\ny = data[-1:, :]\nprint("\n The split out input and output elements are: ", X.shape, y.shape)\n# split into train test sets\n\nX_train, X_test = train_test_split(X, test_size=0.33, random_state=1)\nprint("\n The elements sorted out into train and test sets are: ", X_train.shape, X_test.shape)'

In [17]:
# Splitting of data into train and test set.Generally we have to take 80% for training and 20% for testing.
i_cols = ['user_id', 'post_id', 'timestamp']
views_train = pd.read_csv('/content/views.csv')
views_test = pd.read_csv('/content/views.csv')
views_train.shape, views_test.shape

#This following snippets prints the number of unique
n_users = views.user_id.unique().shape[0]
n_posts = views.post_id.unique().shape[0]

print("\n Number of unique users are: ",n_users)
print("\n Number of unique views are: ",n_posts)

#views.drop(['timestamp'], axis=1,inplace=True)---> we can do this if we want to as the timestamp is redundant in this recommendation engine


 Number of unique users are:  118

 Number of unique views are:  495


In [ ]:
#This is similairty matrix which finds akin data in the given data sets.
data_matrix = np.zeros((n_users, n_posts))
for line in views.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

In [19]:
# This cell prints the similarity matrices of the users and posts data provided using cosine similairty 
#cosine similairty would be the perfect one for this engine.
user_similarity = pairwise_distances(data_matrix, metric='cosine')
post_similarity = pairwise_distances(data_matrix.T, metric='cosine')

print("The user similarity is: \n",user_similarity)
print("\nThe view similarity is: \n",post_similarity)

The user similarity is: 
 [[0. 1. 1. ... 1. 1. 1.]
 [1. 0. 1. ... 1. 1. 1.]
 [1. 1. 0. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]]

The view similarity is: 
 [[0. 1. 1. ... 1. 1. 1.]
 [1. 0. 1. ... 1. 1. 1.]
 [1. 1. 0. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]]


In [20]:
# The following code does predcitions based on the similarities gauged in the above cell.
def predict(views, similarity, type='users'):
    if type == 'users':
        mean_user_views = views.mean(axis=1)
#We use np.newaxis so that mean_user_rating has same format as views
        views_diff = (views - mean_user_views[:, np.newaxis])
        pred = mean_user_views[:, np.newaxis] + similarity.dot(views_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'posts':
        pred = views.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        return pred
    
user_prediction = predict(data_matrix, user_similarity, type='users')
post_prediction = predict(data_matrix, post_similarity, type='posts')

In [21]:
train_data = turicreate.SFrame(views_train)
test_data = turicreate.SFrame(views_test)

#Training the machine with the train data which is defined just above this comment.
item_sim_model = turicreate.item_similarity_recommender.create(train_data, user_id='user_id', item_id='post_id', target='', similarity_type='cosine')

# Now the machine recommends ten posts each for the first ten users as we have mentioned users = 10 and k = 10.
# we can change these values to check the for the entire data which would consume more time.
item_sim_recomm = item_sim_model.recommend(users=[1,2,3,4,5,6,7,8,9,10],k=10)
item_sim_recomm.print_rows(num_rows=50)

Warning: Ignoring columns timestamp;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 1449 observations with 118 users and 495 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.018649s

+---------+--------------------------+---------------------+------+
| user_id |         post_id          |        score        | rank |
+---------+--------------------------+---------------------+------+
|    1    | 5ec57acaf2781131cc7e51d5 |  0.2554840636253357 |  1   |
|    1    | 5ec1fd0974f7660d73aa0fd5 |  0.2380655264854431 |  2   |
|    1    | 5ec54fc9f2781131cc7e50f5 | 0.23015980124473573 |  3   |
|    1    | 5ec7a8bdec493f4a26558846 | 0.22907609701156617 |  4   |
|    1    | 5ec2215374f7660d73aa1011 | 0.22455269932746888 |  5   |
|    1    | 5ec3c09274f7660d73aa1229 | 0.22090505361557006 |  6   |
|    1    | 5ec8204cec493f4a26558893 | 0.22080119609832763 |  7   |
|    1    | 5ec7a7a3ec493f4a26558840 | 0.21719491362571716 |  8   |
|    1    | 5ecd5d417023451e662235c5 |  0.2166977369785309 |  9   |
|    1    | 5ec821ddec493f4a2655889e | 0.20753011465072632 |  10  |
|    2    | 5ec57acaf2781131cc7e51d5 |  0.2554840636253357 |  1   |
|    2    | 5ec1fd0974f7660d73aa0fd5 |  0.238065